In [3]:
# Bokeh libraries
import bokeh
import pandas as pd
import numpy as np
import os
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Select, Slider, TextInput
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel

In [4]:
path = os.listdir('./data/')

cfrm_g = pd.read_csv('./data/' + path[0] , index_col=1+0)
cfrm_reg = pd.read_csv('./data/' + path[1])
cfrm_narrow = pd.read_csv('./data/' + path[2])

death_g = pd.read_csv('./data/' + path[3], index_col=1)
death_reg = pd.read_csv('./data/' + path[4])
death_narrow = pd.read_csv('./data/' + path[5])

recov_g = pd.read_csv('./data/' + path[6], index_col=1)
recov_reg = pd.read_csv('./data/' + path[7])
recov_narrow = pd.read_csv('./data/' + path[8])

In [6]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, RadioButtonGroup, Button, Toggle

from bokeh.layouts import column, row, WidgetBox, layout
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.transform import dodge

output_notebook()

def modify_doc(doc) :
    #DATA PREPARATION
    re_index = lambda x: [x.iloc[i+1][1] if (type(x.iloc[i+1][0]) ==float) else (str(x.iloc[i+1][1])+' ('+str(x.iloc[i+1][0])+')') for i in range(len(x[x.columns[0]])-1)]
    cfrm_g.index = re_index(cfrm_reg)
    recov_g.index = re_index(recov_reg)
    death_g.index = re_index(death_reg)
    years = cfrm_g.columns[3:].tolist()
    status = ['recovered', 'confirmed', 'death']
    colors = [ "green", "blue", "red", 'orange']
    s = [(status) for i in range(86)]
    x = pd.to_datetime(years)

    #SET WIDGETS
    slider_month = Slider(title ='Month', value=0, start=0, end=4, step=1, height=1, width=10)
    button = Button(label="Show Additional disease per day", button_type="success")

    region = Select(title="Country/Region", value="Afghanistan",
               options= cfrm_g.index.tolist(), height=1, width=10)
    
    RB = RadioButtonGroup(labels=["Show All", "Hide Recovered", "Hide Confirm", "Hide Death"], active=0)
    
    
    #FUNCTION UPDATE DATA PLOTTING
    get_month = lambda x,y : [y for s in range(y) for y in years if int(y[0])==s+1 ] #FUNGSI UNTUK MENDAPATKAN PANJANG DATE YG DIINPUTKAN
    def select_region():
        #GET VALUE UPDATE
        region_val = region.value
        month = slider_month.value
        
        if slider_month.value != 0 : #CHECK SLIDER MONTH
            month = get_month(years, month)
        else :
            month = years
            
        if (region_val != 'All'): #CHECK SELECTED REGION
            selected = dict(Confirm = cfrm_g[month].loc[region_val].tolist(),
                             Recovered = recov_g[month].loc[region_val].tolist(),
                                Death = death_g[month].loc[region_val].tolist(),
                            Month = month)
        
        if (RB.active != 0): #CHECK RADIO BUTTON GROUP
            selected[RB.labels[RB.active].split()[1]] = np.zeros(86) #MERUBAH NILAI MENJADI 0 UNTUK BUTTON YG ACTIVE
        return selected
    
    def global_virus() :
        month = slider_month.value 
        
        if slider_month.value != 0 : #CHECK SLIDER MONTH
            month = get_month(years, month)
        else :
            month = years
        
        g_month = lambda data,months : [data[i].sum() for i in months] #FUNGSI UNTUK MENDAPATKAN TOTAL CASE PADA SATU HARI
        selected = dict(Confirm = g_month(cfrm_g, month),
                         Recovered = g_month(recov_g, month),
                         Death = g_month(death_g, month),
                         Month = month)
        
        if (RB.active != 0):
            selected[RB.labels[RB.active].split()[1]] = np.zeros(86)
        
        return selected
    
    def stacked_data() :
        region_val = region.value
        month = slider_month.value

        if slider_month.value != 0 : #CHECK VALUE SLIDER MONTH
            month = get_month(years, month)
        else :
            month = years
            
        if (region_val != 'All'): #CHECK SELECTED REGION
            add = lambda x : [[0, 0] if i==0 else [x[i]-x[i-1], x[i-1]] for i in range(len(x))] #FUNGSI MENDAPATKAN SELISIH NILAI KASUS HARI INI DAN KEMARIN
            
            cfrm = add(cfrm_g[month].loc[region_val])
            recov = add(recov_g[month].loc[region_val])
            death = add(death_g[month].loc[region_val])
            
            selected = dict(Confirm = [i[1] for i in cfrm],
                             Recovered = [i[1] for i in recov],
                                Death = [i[1] for i in death],
                            add_confirmed = [i[0] for i in cfrm],
                            add_recov = [i[0] for i in recov],
                            add_death = [i[0] for i in death],
                            Month = month)
        if (RB.active != 0): #RAGIO BUTTON CHECKED
            selected[RB.labels[RB.active].split()[1]] = np.zeros(86)
        return selected
    
    #GET Data
    data = select_region()
    data_g = global_virus()
    data_stack = stacked_data()

    #SET FIGURE
    p = figure(x_range=years,y_range=(0,1000), 
               plot_width=720, plot_height=500,  
               title='Region %s Covid-19' %region.value, tools="crosshair,pan,reset,save,wheel_zoom")
    
    p_stack = figure(x_range=years,y_range=(0,1000), 
               plot_width=720, plot_height=500,  
               title='Region %s Covid-19' %region.value, tools="crosshair,pan,reset,save,wheel_zoom")
   
    p_g = figure(x_range=years,y_range=(0,int(max(data_g['Confirm'])+1)), 
               plot_width=720, plot_height=500,  
               title='Global Covid-19' , tools="crosshair,pan,reset,save,wheel_zoom",
               x_axis_type="datetime" )
    
    
    #CREATE DATA SOURCES UNTUK PLOTTING
    source = ColumnDataSource(data=dict(x=years, recovered = data['Recovered'],
                                        confirmed = data['Confirm'],
                                        death= data['Death'], status=s))
    
    source_stack = ColumnDataSource(data=dict(x=years, recovered = data_stack['Recovered'],
                                        confirmed = data_stack['Confirm'],
                                        death= data_stack['Death'],
                                        add_recovered = data_stack['add_recov'],
                                        add_confirmed = data_stack['add_confirmed'],
                                        add_death = data_stack['add_death'],
                                        status=s))
    
    source_g = ColumnDataSource(data=dict(x=years, recovered = data_g['Recovered'],
                                        confirmed = data_g['Confirm'],
                                        death= data_g['Death'], status=s))
    
    #DATA PLOTTING LINE 
    p_g.line(x='x', y='recovered', line_width=2, source=source_g, legend_label="Recovered", color='green', name='recovered')
    p_g.line(x='x', y='confirmed', line_width=2, source=source_g, legend_label="Confirmed", color='blue', name='confirmed')
    p_g.line(x='x', y='death', line_width=2, source=source_g, legend_label="Death", color='red', name='death')
    
    #DATA PLOTTING VBAR
    p.vbar(x=dodge('x', -0.3, range=p.x_range), top='recovered', bottom=0,
           width=0.3, color=colors[0], source = source, name='recovered', legend_label="Recovered")
    p.vbar(x=dodge('x', 0., range=p.x_range), top='confirmed', bottom=0,
           width=0.3, color=colors[1], source = source, name='confirmed', legend_label="Confirmed")
    p.vbar(x=dodge('x', 0.3, range=p.x_range), top='death', bottom=0,
           width=0.3, color=colors[2], source = source, name='death', legend_label="Death")
    
    #DATA PLOTTING STACKED VBAR
    stack_1 = p_stack.vbar_stack(['recovered', 'add_recovered'], x=dodge('x', -0.3, range=p.x_range ),
                       width=0.3, color=[colors[0], colors[3]], source=source_stack,
                         legend_label=['recovered', 'additional case per day'], name=['recovered','add_recovered'])
    stack_2 = p_stack.vbar_stack(['confirmed', 'add_confirmed'], x=dodge('x', 0., range=p.x_range ),
                       width=0.3, color=[colors[1], colors[3]], source=source_stack,
                         legend_label=['confirmed', 'additional case per day'], name=['confirmed','add_confirmed'])
    stack_3 = p_stack.vbar_stack(['death', 'add_death'], x=dodge('x', 0.3, range=p.x_range ),
                       width=0.3, color=[colors[2], colors[3]], source=source_stack,
                         legend_label=['death', 'additional case per day'], name=['death','add_death'])
    
    #SET HOVER TOOL FOR STACKED VBAR PLOT
    for render in stack_1 :
        name = render.name
        hover = HoverTool(tooltips=[
                                 ("Number", "@$name"),("Date ", "@x"), ('Status', '$name')
                            ], renderers=[render])
        p_stack.add_tools(hover)
    for render in stack_2 :
        name = render.name
        hover = HoverTool(tooltips=[
                                 ("Number", "@$name"),("Date ", "@x"), ('Status', '$name')
                            ], renderers=[render])
        p_stack.add_tools(hover)
    for render in stack_3 :
        name = render.name
        hover = HoverTool(tooltips=[
                                 ("Number", "@$name"),("Date ", "@x"), ('Status', '$name')
                            ], renderers=[render])
        p_stack.add_tools(hover)
    

    #SET HOVER TOOL FOR GLOBAL AND REGION PLOT
    hover_tool = HoverTool(tooltips=[
        ("Number", "@$name"), ("Date ", "@x"), ('Status', '$name')
    ], mode='vline')
    p_g.tools.append(hover_tool)
    p.tools.append(hover_tool)
    
    #SET XAXIS LABEL ORIENTATION
    p.xaxis.major_label_orientation = -1.3
    p_g.xaxis.major_label_orientation = -1.3
    p_stack.xaxis.major_label_orientation = -1.3
    
    #SET LEGEND ORIENTATION, LOCATION, AND CLICK ACTION
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.legend.click_policy="hide"
    p_g.legend.location = "top_left"
    p_g.legend.orientation = "horizontal"
    p_g.legend.click_policy="hide"
    p_stack.legend.location = "top_left"
    p_stack.legend.orientation = "horizontal"
    p_stack.legend.click_policy="hide"
    
    #Update Data PLOTTING
    def update():
        new_data = select_region()
        new_data_g = global_virus()
        new_data_stack = stacked_data()
        
        #SET NEW DATA REGION PLOTTING VBAR 
        source.data = dict( x = new_data['Month'], recovered = new_data['Recovered'], 
                           confirmed =new_data['Confirm'], death = new_data['Death'],
                           status = s[:len(new_data['Month'])])
        
        #SET NEW DATA GLOBAL DISEASE PLOTTING LINE
        source_g.data = dict( x = new_data_g['Month'], recovered = new_data_g['Recovered'], 
                           confirmed =new_data_g['Confirm'], death = new_data_g['Death'],
                           status = s[:len(new_data_g['Month'])])
        
        #SET NEW DATA ADDITIONAL CASE PER DAY PLOTTING STACKED VBAR 
        source_stack.data = dict( x = new_data_stack['Month'], recovered = new_data_stack['Recovered'], 
                           confirmed =new_data_stack['Confirm'], death = new_data_stack['Death'],
                             add_recovered = new_data_stack['add_recov'], add_confirmed = new_data_stack['add_confirmed'],
                             add_death = new_data_stack['add_death'],
                           status = s[:len(new_data_stack['Month'])])
        #SUPDATE TITLE PLOT
        p.title.text ='Region %s Covid-19' %region.value
    
    #MONITORING WIDGETS
    controls = [region, slider_month]
    for control in controls :
        control.on_change('value', lambda attr, old, new: update())
    RB.on_change('active', lambda attr, old, new: update())
    
    #LAYOUTING
    widgets = layout(children=(region, slider_month, RB), sizing_mode='scale_width')
    layout_1 = layout([[ p]], sizing_mode='scale_width')
    layout_2 = layout([[ p_g]], sizing_mode='scale_width')
    layout_3 = layout([[ p_stack]], sizing_mode='scale_width')
    tab_1 = Panel(child=layout_1, title='Covid-19 Regions')
    tab_2 = Panel(child=layout_2, title='Covid-19 Global')
    tab_3 = Panel(child=layout_3, title='Covid-19 Regions additional virus')
    tabs = Tabs(tabs=[tab_1, tab_2, tab_3])
    l = column(widgets , tabs)
    
    doc.add_root(l)
show(modify_doc)

Loading BokehJS ...